In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils import weight_norm
from torch.utils.data import DataLoader, Dataset

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import average_precision_score
from sklearn.metrics import r2_score

In [3]:
import logging

logger = logging.getLogger('rnn')
logger.setLevel(logging.DEBUG)
# create file handler which logs even debug messages
fh = logging.FileHandler('ss.log')
fh.setLevel(logging.INFO)
# create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
fh.setFormatter(formatter)
# add the handlers to logger
logger.addHandler(ch)
logger.addHandler(fh)

In [4]:
data_whole = pd.read_csv('../Data/ground_truth/ground_truth.csv')

In [ ]:
class Data(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.labels_h_up) - 100

    def __getitem__(self, index):
        item = self.data[index]
        l_h_up = self.labels[index + 100]

        return item, l_h_up

In [ ]:
if case == 1: 
    sc = MinMaxScaler(feature_range = (-1,1))
    train_data = sc.fit_transform(train_data)
    val_data = sc.fit_transform(val_data)
    test_data = sc.fit_transform(test_data)

In [ ]:
if case == 1:
    train_data = torch.tensor(train_data)
    val_data = torch.tensor(val_data)
    test_data = torch.tensor(test_data)
    print("scaled data")
if case == 0:
    train_data = torch.tensor(train_data.values)
    val_data = torch.tensor(val_data.values)
    test_data = torch.tensor(test_data.values)
    print("not scaled data")

In [ ]:
train_data_h = Data(train_data, train_l_h_merged)
test_loader_d = DataLoader(test_data_d, batch_size=64)

In [ ]:
''' New activation function '''

class Mish(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        x = x * (torch.tanh(F.softplus(x)))
        return x

In [ ]:
''' Defining neural net '''

class Linear_out(nn.Module):
    def __init__(self, in_size, out_size):
        super().__init__()

        self.dense_1 = nn.Sequential(
            nn.Linear(in_size, 64),
            Mish(),
            nn.Dropout(p=0.2)
        )
        self.dense_2 = nn.Sequential(
            nn.Linear(64, 10), 
            nn.Dropout(p=0)
        )
        self.dense_3 = nn.Linear(10, 1)

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.dense_1(x)
        print("after dense 1: ", x[0])
        x = self.dense_2(x)
        print("after dense 2: ", x[0])
        print("sanity check: ", x[0]-x[1]+x[2]-x[3])
        x = self.dense_3(x)
        print("after dense 3: ", x[:7])
        return x

In [ ]:
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

In [ ]:
class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, dilation, dropout = 0.2):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation, padding_mode='zeros')
        self.group_norm1 = nn.GroupNorm(out_channels, out_channels)
        self.chomp1 = Chomp1d(padding)
        self.mish1 = Mish()
        self.dropout1 = nn.Dropout(dropout)


        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation, padding_mode='zeros')
        self.group_norm2 = nn.GroupNorm(out_channels, out_channels)
        self.chomp2 = Chomp1d(padding)
        self.mish2 = Mish()
        self.dropout2 = nn.Dropout(dropout)

        self.net_1 = nn.Sequential(self.conv1, self.chomp1, self.group_norm1, self.mish1, self.dropout1)
        self.net_2 = nn.Sequential(self.conv2, self.chomp2, self.group_norm2, self.mish2, self.dropout2)
        self.mish = Mish()

        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None

        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        out = self.net_1(x)
        # print("conv_output_1: ", out.size()) 
        out = self.net_2(out)
        # print("conv output_2: ", out.size()) 
        res = self.downsample(x)
        # print("conv downsample: ", res.size()) 
        return self.mish(res + out)

In [ ]:
class TemporalConvNet(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=4, dilation_size=1):
        super(TemporalConvNet, self).__init__()

        self.group_norm1 = nn.GroupNorm(in_channels, in_channels)
        self.group_norm2 = nn.GroupNorm(out_channels, out_channels)
        self.mish1 = Mish()
        self.conv_part = nn.Sequential(TemporalBlock(in_channels, 32, kernel_size, stride=1, dilation=dilation_size,
                                    padding=kernel_size-1),
                                    TemporalBlock(32, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                    padding=kernel_size-1))
        self.linear_part = Linear_out(out_channels*100, 1)

    def forward(self, x):
        x = self.group_norm1(x)
        x = self.conv_part(x)
        print("After conv part:", x[0])
        test = self.mish1(x)
        print("After mish without group norm: ", test[0])
        x = self.group_norm2(x)
        x = self.mish1(x)
        print("After mish with group norm: ", x[0])
        x = torch.flatten(x, start_dim=1)
        x = self.linear_part(x)
        return x.view(-1)

In [ ]:
''' Eval function '''

def Eval_epoch(model, dataloader, criterion, epoch):
    losses = []

    run_mean = []
    
    preds = {
        "h_up":[]
    }
    labels = {
        "h_up":[]
    }

    with torch.no_grad():
        for step, batch in enumerate(dataloader):
            seqs, label_h_up = batch
            h_up = model(seqs)
            preds["h_up"].append(h_up.cpu().numpy())
            labels["h_up"].append(label_h_up.cpu().numpy())

            loss = criterion(h_up.float(), label_h_up.float())

            losses.append(loss)

            run_mean.append(sum(losses)/len(losses))

            if step % 20 ==0:
                logger.info("Eval epoch:{}    Step:{}/{}    loss:{:.4f}({:.4f})".format(epoch, step, len(dataloader), loss, sum(losses)/len(losses)))
                logger.debug("Output : {}    Label : {}".format(h_up, label_h_up))

    final_loss = sum(losses)/len(losses)
    logger.info("Final loss: {:.4f}".format(final_loss))

    preds["h_up"] = np.concatenate(preds["h_up"])

    labels["h_up"] = np.concatenate(labels["h_up"])

    logger.info("r2 score: {:.5f}".format(r2_score(labels['h_up'], preds['h_up'])))

    return preds, labels, run_mean

In [ ]:
''' Initiation of TCN '''

net = TemporalConvNet(259, 64)
net.double()
if torch.cuda.is_available():
    net.cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.01)
epochs=10

In [ ]:
''' Training cell for TCN '''

loss_history = [None]*10
for epoch in range(10):
    loss_history[epoch] = []
    net.train()
    for step, batch in enumerate(train_loader_d):
        input_data, labels = batch
        out = net(input_data)
        loss = criterion(out, labels)
        loss_history[epoch].append(loss)
        optimizer.zero_grad()
        loss.backward()
        if step % 40 == 0:
            logger.info("Train epoch: {}    Step: {}/{}     mse: {}({})     prediction range: {}".format(epoch, step, len(train_loader_d), loss, sum(loss_history[epoch])/len(loss_history[epoch]), max(out)-min(out)))
            print("Output: {}".format(out))
            print("Labels: {}".format(labels))
        optimizer.step()
    # net.eval() 
    predictions, labels, run_mean = Eval_epoch(net, val_loader_d, criterion, epoch)
    torch.save({
        'epoch': epoch,
        'model_state_dict': net.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        }, 'drive/My Drive/Auto ML/TCN_d_full_{}.pt'.format(epoch))


In [ ]:
''' Load and test TCN '''
net = TemporalConvNet(259, 64).double()
if torch.cuda.is_available:
    net.cuda()
    map_location = None
else:
    map_location = 'cpu'
optimizer = torch.optim.Adam(net.parameters())

checkpoint = torch.load('drive/My Drive/Auto ML/TCN_d_full_4.pt', map_location=map_location)
net.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']

criterion = nn.MSELoss()

In [ ]:
preds, labels, run_means = Eval_epoch(net, test_loader_d, nn.MSELoss(), epoch) 

In [ ]:
plt.scatter(range(len(labels["h_up"])), labels["h_up"]-preds["h_up"], s=0.5)
plt.show()